In [95]:
import os
import tweepy as tw
import pandas as pd
import csv
import pandas as pd
import glob
from nltk.corpus import stopwords
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from sklearn.model_selection import train_test_split
import keras.models
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras import backend as K

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
extension = 'csv'
all_filenames = [i for i in glob.glob('/content/drive/MyDrive/twit/train/*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "aggregate.csv", index=False, encoding='utf-8-sig')

In [98]:

names = ['TweetID', 'Sentiment', 'Tweet']

train = pd.read_csv('/content/drive/MyDrive/twit/train/aggregate.csv')

train.head()

,Column1,Column2,Column3
0,6.380610e+17,neutral,05 Beat it - Michael Jackson - Thriller (25th ...
1,6.380610e+17,positive,Jay Z joins Instagram with nostalgic tribute t...
2,6.380840e+17,neutral,Michael Jackson: Bad 25th Anniversary Edition ...
3,6.380910e+17,positive,I liked a @YouTube video http://t.co/AaR3pjp2P...
4,6.381260e+17,positive,18th anniv of Princess Diana's death. I still ...


In [99]:
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)        
  return input_txt

def clean_tweets(frame, column_name):
  frame = frame.drop_duplicates().reset_index(drop=True) #remove duplicate rows
  frame['Tweet_Clean_Text'] = np.vectorize(remove_pattern)(frame[column_name], "RT @[\w]*:") #remove twitter return handle
  frame.Tweet_Clean_Text = np.vectorize(remove_pattern)(frame['Tweet_Clean_Text'], "@[\w]*") #remove twitter handle
  frame.Tweet_Clean_Text = np.vectorize(remove_pattern)(frame['Tweet_Clean_Text'], "https?://[A-Za-z0-9./]*") #remove URLs
  frame.Tweet_Clean_Text = frame.Tweet_Clean_Text.str.replace("[^a-zA-Z#]", " ") #remove special characters except for #
  frame.Tweet_Clean_Text = frame.Tweet_Clean_Text.replace('\s+', ' ', regex=True) #remove extra spaces in between words

  return frame

In [100]:
train_set = clean_tweets(train, 'Column3')
train_set.head()

,Column1,Column2,Column3,Tweet_Clean_Text
0,6.380610e+17,neutral,05 Beat it - Michael Jackson - Thriller (25th ...,Beat it Michael Jackson Thriller th Anniversa...
1,6.380610e+17,positive,Jay Z joins Instagram with nostalgic tribute t...,Jay Z joins Instagram with nostalgic tribute t...
2,6.380840e+17,neutral,Michael Jackson: Bad 25th Anniversary Edition ...,Michael Jackson Bad th Anniversary Edition Pic...
3,6.380910e+17,positive,I liked a @YouTube video http://t.co/AaR3pjp2P...,I liked a video One Direction singing Man in t...
4,6.381260e+17,positive,18th anniv of Princess Diana's death. I still ...,th anniv of Princess Diana s death I still wa...


In [101]:
train_set["Sentiment_Value"] = train_set["Column2"].map({"neutral": 0, "positive": 1, "negative": 2})
label = to_categorical(train_set["Sentiment_Value"], 3)
train_set

,Column1,Column2,Column3,Tweet_Clean_Text,Sentiment_Value
0,6.380610e+17,neutral,05 Beat it - Michael Jackson - Thriller (25th ...,Beat it Michael Jackson Thriller th Anniversa...,0
1,6.380610e+17,positive,Jay Z joins Instagram with nostalgic tribute t...,Jay Z joins Instagram with nostalgic tribute t...,1
2,6.380840e+17,neutral,Michael Jackson: Bad 25th Anniversary Edition ...,Michael Jackson Bad th Anniversary Edition Pic...,0
3,6.380910e+17,positive,I liked a @YouTube video http://t.co/AaR3pjp2P...,I liked a video One Direction singing Man in t...,1
4,6.381260e+17,positive,18th anniv of Princess Diana's death. I still ...,th anniv of Princess Diana s death I still wa...,1
...,...,...,...,...,...
114213,6.390170e+17,neutral,@YouAreMyArsenal Wouldn't surprise me if we en...,Wouldn t surprise me if we enquired He can t ...,0
114214,6.402770e+17,neutral,Rib injury for Zlatan against Russia is a big ...,Rib injury for Zlatan against Russia is a big ...,0
114215,6.402970e+17,neutral,Noooooo! I was hoping to see Zlatan being Zlat...,Noooooo I was hoping to see Zlatan being Zlata...,0
114216,6.410170e+17,neutral,@Fronsoir Zlatan has never done it on a wet Tu...,Zlatan has never done it on a wet Tuesday nig...,0


In [102]:
y_labels = train_set['Sentiment_Value']
y_labels.shape

(114218,)

In [103]:
sequence_length = 47 #using the maximum length 

In [104]:
max_features = 2500
tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(train_set['Tweet_Clean_Text'].values)

X = tokenizer.texts_to_sequences(train_set['Tweet_Clean_Text'].values)
X = pad_sequences(X, sequence_length)
x_train = X

print("training size " + str(len(x_train)))

voc_size = len(tokenizer.word_index) + 1
print("Vocab size: ", voc_size)
print("Input shape: ", x_train.shape)
print("Y_shape: " , y_labels.shape)

training size 114218
Vocab size:  63987
Input shape:  (114218, 47)
Y_shape:  (114218,)


In [105]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_labels, test_size=0.10, shuffle=False, random_state=10)

In [106]:
def recall_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    recall_score.__name__ = 'recall'
    return recall

def precision_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    precision_score.__name__ = 'precision'
    return precision


def f1_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1_metrics.__name__ = 'f1'
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

metrics = [
           keras.metrics.CategoricalAccuracy(),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           f1_metrics,
           #keras.metrics.TruePositives(name='tp'),
           #keras.metrics.FalsePositives(name='fp'),
           #keras.metrics.TrueNegatives(name='tn'),
           #keras.metrics.FalseNegatives(name='fn'),
]


In [107]:
# stacked generalization with neural net meta model on blobs dataset
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers.merge import concatenate
from numpy import argmax

# load models from file
def load_all_models(model_names):
	all_models = list()
	for model_name in model_names:
		# define filename for this ensemble
		filename = '/content/drive/MyDrive/Models/' + model_name
		# load model from file
		model = load_model(filename,custom_objects={'f1':f1_metrics,'precision':precision_score,'recall':recall_score})
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

# load all models
members = ['LSTM532.h5', 'LSTM456.h5', 'LSTM352.h5', 'LSTM256.h5', 'LSTM200.h5']
members = load_all_models(members)
print('Loaded %d models' % len(members))

train_predictions = []
test_predictions = []

for model in members:
  train_predictions.append(model.predict([x_train], batch_size=1024))
  test_predictions.append(model.predict([x_test], batch_size=1024))
  print('Predicted one thing')

>loaded /content/drive/MyDrive/Models/LSTM532.h5
>loaded /content/drive/MyDrive/Models/LSTM456.h5
>loaded /content/drive/MyDrive/Models/LSTM352.h5
>loaded /content/drive/MyDrive/Models/LSTM256.h5
>loaded /content/drive/MyDrive/Models/LSTM200.h5
Loaded 5 models
Predicted one thing
Predicted one thing
Predicted one thing
Predicted one thing
Predicted one thing


In [108]:
from keras.layers import BatchNormalization
def ensemble(predictions, targets):
    layer_size = len(predictions)
    inp = Input(shape=(layer_size,3))
    f0 = Flatten()(inp)
    d0 = Dropout(0.2)(f0)
    d0 = Dense(pow(layer_size, 4))(d0)
    d1 = Dropout(0.2)(d0)
    d1 = Dense(20 * layer_size)(d1)
    b = BatchNormalization()(d1)
    out = Dropout(0.2)(b)
    out = Dense(3, activation='sigmoid')(out)

    model = Model(inputs=inp, outputs=out)
    metrics = [
           keras.metrics.CategoricalAccuracy(),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           f1_metrics,
           #keras.metrics.TruePositives(name='tp'),
           #keras.metrics.FalsePositives(name='fp'),
           #keras.metrics.TrueNegatives(name='tn'),
           #keras.metrics.FalseNegatives(name='fn'),
  ]
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=metrics)
    model.summary()

    x = np.array(list(zip(*np.squeeze(predictions))))
    y = targets
    print(np.shape(x))
    print(np.shape(y))
    
    history_flat_CNN = model.fit(x=x, y=y, epochs=20, validation_split=0.05, verbose=2)
    #display_model_history(history_flat_CNN) 
    #plot_metrics(history_flat_CNN)
    return model, history_flat_CNN

In [109]:
y_train_cat = to_categorical(y_train,3)
y_dev_cat = to_categorical(y_test,3)    
stack_model, history = ensemble(train_predictions, y_train_cat)

stacked_test_predictions = np.array(list(zip(*np.squeeze(test_predictions))))
stacked_test_predictions = stack_model.predict(stacked_test_predictions, batch_size=1024)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 5, 3)]            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 15)                0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 15)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 625)               10000     
_________________________________________________________________
dropout_13 (Dropout)         (None, 625)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 100)               62600     
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400 

In [110]:
indiv = []
for model in members:
  indiv.append(f1_metrics(y_dev_cat, model.predict(x_test)))

max(indiv)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6822533>

In [111]:
stack_model.evaluate(np.array(list(zip(*np.squeeze(test_predictions)))),y_dev_cat)

357/357 [==============================] - 1s 3ms/step - loss: 0.7353 - categorical_accuracy: 0.6884 - precision: 0.6374 - recall: 0.7746 - f1_metrics: 0.7001


[0.7353222966194153,
 0.6884083151817322,
 0.6373721361160278,
 0.7746454477310181,
 0.7001225352287292]